In [1]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import pandas as pd
from sqlalchemy import create_engine

## connection setup

#### neo4j

In [10]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "Ruifan0511"))
gds = GraphDataScience(uri, auth=("neo4j", "Ruifan0511"))

In [17]:
def show_result(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame(result.data())
    #print(result.data())

In [15]:
def delete_graph_if_exists(graph_name):
    with driver.session() as session:
        try:
            existing_graph = gds.graph.get(graph_name)
            if existing_graph is not None:
                gds.graph.drop(graph_name)
                #print(f"Graph '{graph_name}' dropped.")
        except Exception as e:
            print(f"Graph '{graph_name}' does not exist or could not be retrieved. Nothing to delete.")

#### postgreSQL

In [4]:
db_user = "postgres"
db_password = "mf9500"
db_host = "localhost"  
db_port = "5432"  
db_name = "research_papers"

engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as connection:
    print("connected")

connected


### most influential paper per topic per year
- use `PageRank` to quanify a paper's relevence per topic per year
- basically finds the paper that has become the most 'popular' amongst papers that were published in the same year about the same category

Most Influential Paper by Topic and Year
This query uses PageRank to measure a paper’s relevance and influence within its topic for a given year. By analyzing citation patterns, it identifies the paper that has gained the most recognition and impact among others published in the same year within the same category.

Why is this insightful?
Academic influence is often context-dependent—a paper’s impact is most meaningful when compared to its peers in the same field and time period. This query helps highlight research that has significantly shaped discussions within its specific academic community, rather than just globally.

In [28]:
query = """
    SELECT topic, year, array_agg(id) as paper_ids 
    FROM papers 
    GROUP BY topic, year
    ORDER BY topic, year;
"""

df = pd.read_sql(query, engine)
df

,topic,year,paper_ids
0,0,2010,"[4af00d38-e8a2-4f4c-a819-816772c52e0f, 4c03835..."
1,0,2011,"[4abddc96-3f8c-4f2b-a83b-4c1a6fa1e3d5, 4ae0227..."
2,0,2012,"[4ad24813-c42d-4807-8bae-f8aca28d259c, 4b3ae4c..."
3,0,2013,"[4b8f6032-3d31-421f-a46d-63a339f80012, 4bb890f..."
4,0,2014,"[4acb3623-e94f-404d-a122-bc92f3ee654f, 4ae0c0c..."
...,...,...,...
115,14,2013,"[4b71d6bf-5ffb-49d8-a621-c70ed5cf0fbd, 4b76062..."
116,14,2014,"[4ae093d8-9359-4783-86a6-c279afe38a21, 4af911e..."
117,14,2015,"[4b3b22c6-c3a2-455d-b189-bb8bf280dfe0, 4b8b0ad..."
118,14,2016,"[4acd8824-2bdc-4c08-b4a6-cb5cefbd0379, 4c16d3c..."


In [29]:
results = []
    
with driver.session() as session:
    for index, row in df.iterrows():
        topic = row["topic"]
        year = row['year']
        paper_ids = row["paper_ids"] 
        
        # Cypher-friendly list format
        paper_ids_str = "[" + ", ".join(f'"{p}"' for p in paper_ids) + "]"

        query = f"""
        CALL gds.pageRank.stream('citation_graph')
        YIELD nodeId, score
        WHERE gds.util.asNode(nodeId).id IN {paper_ids_str}
        RETURN gds.util.asNode(nodeId).id AS paper_id, score AS influence_score
        ORDER BY influence_score DESC
        LIMIT 1;
        """

        result = session.run(query)
        top_paper = result.single()
        
        if top_paper:
            results.append({
                "topic": topic,
                "year": year,
                "paper_id": top_paper["paper_id"],
                "influence_score": top_paper["influence_score"]
            })

pd.DataFrame(results)

,topic,year,paper_id,influence_score
0,0,2010,9fba111f-d65a-4c21-b02e-a8d4fbbdf92c,5.084531
1,0,2011,cac8070b-570d-4674-b334-dd96bbc223c6,3.400359
2,0,2012,7a18d786-740c-4001-b1db-95f7d7b0ae1a,2.811377
3,0,2013,f9b3409e-ab94-47f4-b926-6702900141e2,2.265877
4,0,2014,f121ff70-19ef-47d8-bc6a-61a7aa201caa,2.261410
...,...,...,...,...
115,14,2013,62c6a9f6-ee1b-4f7f-962c-ea3eb72c0e25,4.458310
116,14,2014,d0c7476b-4267-47ee-8062-4c1437929859,1.644981
117,14,2015,6c6a4710-68a7-4e05-adfb-4bf355c4381d,0.998560
118,14,2016,7f823c31-322c-4525-ba10-7180ae8d7666,0.961240


### self-citation rate per topic
- self-citation rate is calculated as the average \# of times an author cites their own work.
- the self-citation rate is calculated for each topic group
- this gives us insight on which topic is under-researched. we infer that an under-researched field typically has a higher self-citation rate since there are fewer prior works to reference, resulting in a few authors building upon their own previous research instead of integrating other peoples' works.
- *note:* for simplicity, each author is assigned one main topic/field they belong to.

In [57]:
with driver.session() as session:
    query = """
    MATCH (a:Author)-[:AUTHORED]->(p:Paper)-[:CITED]->(p1:Paper)<-[:AUTHORED]-(a)  
    WITH a, COUNT(p1) AS self_citations
    RETURN a.name AS author_name, self_citations
    ORDER BY self_citations DESC
    """
    result = session.run(query).data()
self_citators = pd.DataFrame(result)

In [59]:
self_citators.to_sql("author_self_citations", engine, if_exists="replace", index=False)

175

In [60]:
query = """
WITH author_topic AS (
    SELECT a.name,
           p.topic,
           RANK() OVER(PARTITION BY a.name ORDER BY COUNT(p.topic) DESC) AS rank
    FROM authors AS a
    LEFT JOIN papers AS p ON a.paper_id = p.id
    GROUP BY a.name, p.topic
)
SELECT a.topic, SUM(COALESCE(ac.self_citations, 0)) / COUNT(DISTINCT a.name) AS self_citation_rate
FROM author_topic AS a
LEFT JOIN author_self_citations AS ac ON a.name = ac.author_name
WHERE rank = 1
GROUP BY a.topic
ORDER BY self_citation_rate DESC;
"""

df = pd.read_sql(query, engine)
df

,topic,self_citation_rate
0,2,0.466766
1,5,0.411464
2,8,0.408777
3,11,0.397692
4,13,0.373600
5,7,0.363426
6,10,0.339543
7,1,0.326918
8,6,0.220541
9,3,0.205749


In [61]:
with driver.session() as session:
    query = """
    CALL gds.louvain.stream('citation_graph')
    YIELD nodeId, communityId
    RETURN gds.util.asNode(nodeId).id AS paper_id, communityId
    ORDER BY communityId;
    """
    result = session.run(query).data()
pd.DataFrame(result)

,paper_id,communityId
0,9cfb6867-d5ba-461a-8372-f6f2389e1bde,2
1,9d4f4953-a440-4fb8-bdf4-27786b035e28,5
2,9d54aed2-cb90-4036-bfca-5754508acc92,6
3,9d5a22d1-722a-4495-9747-db9f42e2bd6c,7
4,9dbe1d8c-c9f7-4a19-a92a-4fee395c8e3b,10
...,...,...
134283,9b39472c-cf45-46ca-b223-474e54f267c2,134275
134284,9baada71-05b4-4cee-a561-9b96944e1a5e,134278
134285,9c38a5be-0786-4803-84dc-b9c70e3005a1,134280
134286,9c772bd4-e175-44f0-a75a-f334c6b3935a,134286


In [67]:
com = pd.DataFrame(result)
com['communityId'].value_counts().iloc[:15]

communityId
44102    7432
16299    6726
45063    6487
88249    5863
39576    4654
59454    4218
92849    3803
49081    3680
85030    3664
70388    3588
89973    3201
81391    3127
36688    2402
11102    2334
59322    2299
Name: count, dtype: int64

## ideas
- SQL
    - find in sql the occurrence of certain words (nn)
    - see the trend of the word over time